<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/DeepLearning/3_Redes_Multicapa/2_backpropagation_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Backpropagation y Autograd

In [1]:
import torch

Autograd durante el entrenamiento
--------------------

Hemos echado un breve vistazo a cómo funciona Autograd, pero ¿cómo se ve cuando se usa para el propósito previsto? Definamos un modelo pequeño y examinemos cómo cambia después de un solo lote de entrenamiento. Primero, definimos algunas constantes, nuestro modelo y algunos sustitutos para entradas y salidas:

In [2]:
BATCH_SIZE = 16
DIM_IN = 784
HIDDEN_SIZE = 256
DIM_OUT = 10

net = torch.nn.Sequential(torch.nn.Linear(DIM_IN, HIDDEN_SIZE),
                    torch.nn.ReLU(),
                    torch.nn.Linear(HIDDEN_SIZE, DIM_OUT))
#por defecto en el caso de un modelo secuencial, se calculan los gradientes

# features aleatorias
some_input = torch.randn(BATCH_SIZE, DIM_IN, requires_grad=False)
# etiquetas aleatorias
ideal_output = torch.randn(BATCH_SIZE, DIM_OUT, requires_grad=False)

model = net

Fijemos nos que no hizo falta agregar
``requires_grad=True`` a las capas de modelo esto es por que la clase ``torch.nn.Module`` supone que siempre usaremos el gradiente para entrenar el modelo

Sin embargo, al momento de inicial los valores del modelo, el gradiente no se calcula, hasta que lo pidamos.



In [3]:
print(model[2].weight[0][0:10]) # solo algunos son mostrados
print(model[2].weight.grad)

tensor([-0.0422, -0.0268,  0.0251,  0.0417, -0.0115,  0.0034, -0.0393, -0.0317,
        -0.0144,  0.0118], grad_fn=<SliceBackward0>)
None


Veamos que ocurre ahora si entrenamos.

Consideremos como función de perdida la distancia cuadrática media entre nuestra ``prediction`` y las etiquetas, ``ideal_output``

En este caso usaremos SGD como algoritmos de optimización.




In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

prediction = model(some_input)

loss = (ideal_output - prediction).pow(2).sum()
print(loss)

tensor(143.2557, grad_fn=<SumBackward0>)


Hasta que no llamemos  ``loss.backward()`` los gradientes no se calculan.




In [5]:
print(model[2].weight[0][0:10]) # solo algunos son mostrados
print(model[2].weight.grad)

tensor([-0.0422, -0.0268,  0.0251,  0.0417, -0.0115,  0.0034, -0.0393, -0.0317,
        -0.0144,  0.0118], grad_fn=<SliceBackward0>)
None


In [6]:
loss.backward()
print(model[2].weight[0][0:10])
print(model[2].weight.grad[0][0:10])

tensor([-0.0422, -0.0268,  0.0251,  0.0417, -0.0115,  0.0034, -0.0393, -0.0317,
        -0.0144,  0.0118], grad_fn=<SliceBackward0>)
tensor([ 1.2026,  5.2959, -1.2644, -2.0956, -1.4005,  1.3287,  2.6351,  4.1413,
         0.6922, -4.4739])


Por ahora solo hemos calculados los gradientes, pero no los hemos usada para actualizar los pesos. Esto es porque debemos ejecutar ``optimizer.step()``




In [7]:
print(model[2].weight[0][0:10]) # solo algunos son mostrados
print(model[2].weight.grad[0][0:10])

tensor([-0.0422, -0.0268,  0.0251,  0.0417, -0.0115,  0.0034, -0.0393, -0.0317,
        -0.0144,  0.0118], grad_fn=<SliceBackward0>)
tensor([ 1.2026,  5.2959, -1.2644, -2.0956, -1.4005,  1.3287,  2.6351,  4.1413,
         0.6922, -4.4739])


In [8]:
optimizer.step()
print(model[2].weight[0][0:10])
print(model[2].weight.grad[0][0:10])

tensor([-0.0434, -0.0321,  0.0264,  0.0438, -0.0101,  0.0021, -0.0419, -0.0359,
        -0.0151,  0.0163], grad_fn=<SliceBackward0>)
tensor([ 1.2026,  5.2959, -1.2644, -2.0956, -1.4005,  1.3287,  2.6351,  4.1413,
         0.6922, -4.4739])


Vemos ahora que los valores de ``model[2]`` han cambiado

Un detalle que no dedemos ignorar es que debemos llamar a la función ``optimizer.zero_grad()`` despues de llamar
``optimizer.step()``. De no hacer esto cada vez que llamemos  ``loss.backward()`` la suma de los gradientes se acumulará.




In [9]:
print(model[2].weight.grad[0][0:10])

for i in range(0, 5):
    prediction = model(some_input)
    loss = (ideal_output - prediction).pow(2).sum()
    loss.backward()

print(model[2].weight.grad[0][0:10])

optimizer.zero_grad()


if model[2].weight.grad is not None:
    print(model[2].weight.grad[0][0:10])
else:
  print('Model has no grad')

tensor([ 1.2026,  5.2959, -1.2644, -2.0956, -1.4005,  1.3287,  2.6351,  4.1413,
         0.6922, -4.4739])
tensor([  6.4025,  19.8908, -12.3711, -12.6375,  -4.0476,  -3.3324,   9.5262,
         23.4708,   3.3043, -24.0810])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Contenido adicional: Más información sobre Autograd
-----------------------------------------------------------

En principio, ya conocíamos la noción de gradiente. Sabíamos que para una toma vectores m-dimensionales y devuelve un único valor (un escalar), $l=g\left(\vec{y}\right)$ existe el gradiente. Esto es un vector que nos dice como varía una función conforme cambian los valores del vector de entrada $\vec{y}$


$$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$$

En general, si tenemos una función que toma vectores n-dimensionales como entrada y tiene como salida vectores m-dimensionales, $\vec{y}=f(\vec{x})$, la idea de gradiente no permite abarcar todas las posibles variaciones. En este sentido se necesita una generalización de la idea de gradiente. Esta generalización es una matriz conocida como el
*Jacobiano:*

\begin{align}J
     =
     \left(\begin{array}{ccc}
     \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
     \vdots & \ddots & \vdots\\
     \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
     \end{array}\right)\end{align}

Sin embargo, la función de pérdida nuestros modelos más sencillos son en realidad una combinación de las dos cosas.

$$l=g\left(\vec{y}\right)$$
$$\vec{y}=f(\vec{x})$$
$$l=g\left(f(\vec{x})\right)$$

Puede demostrarse, sin embargo, que para obtener el gradiente de $l$, respecto de $\vec{x}$ solo debemos hacer una multiplicación matricial

$$\vec{\nabla_x} l=J^{T}\cdot v$$

\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

Del mismo modo, a la salida de cada capa, tenemos un Jacobiano distinto. De tal manera que nuestro gradiente en realdiad tendra la forma:

$$\vec{\nabla_x} l=J_{1}^{T} J_{2}^{T} J_{3}^{T} J_{4}^{T}\cdot v$$


**``torch.autograd`` es la herramienta que computa todas estas dependencias por medio de productos matriciales** Además de guardar la relación entre cada salida y cada entrada de cada capa




Para más información consultar

<https://pytorch.org/docs/stable/autograd.html#functional-higher-level-api>